In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Part(b) 
Use an LLM to generate predictions see section 3 of LLMTIME for some ideas on
tokenisation of input, handling null values. Report only your best working prompting strategy
and their evaluations.[You are not expected to train/finetune an LLM nor use multimodal LLM
i.e you input pdf or screenshot, your input to the LLM should be PROMPT + string generated
from timeseries dataframe ideally by a script]

In [2]:
df = pd.read_csv('../air-traffic-prediction/AirtrafficA4.csv')
months = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUNE': '06', 'JULY': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}
#new data frame ndf with just date and passenger carried
ndf = df[['PASSENGERS CARRIED']]
ndf['PASSENGERS CARRIED'] = ndf['PASSENGERS CARRIED'].str.strip().replace(',', '', regex=True)
ndf['PASSENGERS CARRIED'] = ndf['PASSENGERS CARRIED'].astype(float).fillna(0)
ndf['DATE'] = df['YEAR'].astype(str) + '-' + df['MONTH'].str.strip().map(months)
#changing order
ndf = ndf[['DATE', 'PASSENGERS CARRIED']]
ndf.sort_values(by='DATE', inplace=True)
ndf.reset_index(drop=True, inplace=True)
display(ndf)



,DATE,PASSENGERS CARRIED
0,2013-01,1408012.0
1,2013-02,1341210.0
2,2013-03,1423569.0
3,2013-04,1511094.0
4,2013-05,1685168.0
...,...,...
123,2023-04,7406440.0
124,2023-05,8109626.0
125,2023-06,7893296.0
126,2023-07,7674890.0


In [3]:
prompt_text = f'Here, is the dataset of passengers carried by air traffic for the last {ndf.shape[0]} months:\n\nYEAR_MONTH,PASSENGERS CARRIED\n'
prompt_text += '\n'.join([f"{row['DATE']},{row['PASSENGERS CARRIED']}" for _, row in ndf.iterrows()])
next_12_months = []
last_date = ndf['DATE'].iloc[-1]
for i in range(12):
    year, month = last_date.split('-')
    month = int(month) + 1
    if month == 13:
        month = 1
        year = int(year) + 1
    last_date = f"{year}-{str(month).zfill(2)}"
    next_12_months.append(last_date)

prompt_text += '\n'+'\n'.join([f"{date},[forecast_this_month]" for date in next_12_months])
prompt_text += '\n\nBased on the dataset, predict the number of passengers that will be carried by air traffic for the last 12 months.'
prompt_text += '\n\nIgnore the sudden drops that occured due to pandemic so as to get a better prediction.'
prompt_text += '\n\nGive the predicted number of passengers for the next 12 months as a csv file in the following format for year,month: YYYY MON,xxxxxx'

prompt_text += '\n\nSample csv file format'
prompt_text += '\n\nYEAR_MONTH,PASSENGERS CARRIED'
prompt_text += '\n2023 SEP,100000.0'
prompt_text += '\n2023 OCT,100000.0'

print(prompt_text)
with open('Q2b_prompt.txt', 'w') as f:
    f.write(prompt_text)

Here, is the dataset of passengers carried by air traffic for the last 128 months:

YEAR_MONTH,PASSENGERS CARRIED
2013-01,1408012.0
2013-02,1341210.0
2013-03,1423569.0
2013-04,1511094.0
2013-05,1685168.0
2013-06,1480879.0
2013-07,1445248.0
2013-08,1531406.0
2013-09,1378691.0
2013-10,1510184.0
2013-11,1467763.0
2013-12,1575872.0
2014-01,1426580.0
2014-02,1464070.0
2014-03,1601141.0
2014-04,1679963.0
2014-05,1908334.0
2014-06,1797101.0
2014-07,1599977.0
2014-08,1858664.0
2014-09,1907378.0
2014-10,1890273.0
2014-11,1967992.0
2014-12,2324221.0
2015-01,2276404.0
2015-02,2230645.0
2015-03,2286128.0
2015-04,2481285.0
2015-05,2769283.0
2015-06,2536554.0
2015-07,2416916.0
2015-08,2384943.0
2015-09,2430449.0
2015-10,2589861.0
2015-11,2597765.0
2015-12,2743325.0
2016-01,2725711.0
2016-02,2754131.0
2016-03,3023228.0
2016-04,3066556.0
2016-05,3336839.0
2016-06,3023081.0
2016-07,3383768.0
2016-08,3341081.0
2016-09,3295826.0
2016-10,3692828.0
2016-11,3772583.0
2016-12,3848322.0
2017-01,3809228.0
2017

In [4]:
chatGPT_forecast = pd.read_csv('Q2b_prompt_forecast.csv')
display(chatGPT_forecast)

,YEAR_MONTH,PASSENGERS CARRIED
0,2023 SEP,7.873388e+06
1,2023 OCT,8.070753e+06
2,2023 NOV,8.028704e+06
3,2023 DEC,8.199981e+06
4,2024 JAN,7.802780e+06
5,2024 FEB,7.878408e+06
6,2024 MAR,8.303011e+06
7,2024 APR,8.195621e+06
8,2024 MAY,8.498740e+06
9,2024 JUN,8.146103e+06


The score on Kaggle for this is 1.85